In [ ]:
import pathlib
import sys

In [ ]:
import numpy as np
import joblib
import pandas as pd
from tqdm import tqdm

In [ ]:
root_dir = "../.."

In [ ]:
root_dir_path = pathlib.Path(root_dir)
data_dir_path = root_dir_path / "data"
interim_dir_path = data_dir_path / "interim"
interim_path = interim_dir_path / "interim.parquet"
models_dir_path = root_dir_path / "models"
model_path = models_dir_path / "model.joblib"
prediction_path = models_dir_path / "prediction.parquet"
src_dir_path = root_dir_path / "src"

In [ ]:
sys.path.append(str(src_dir_path))

In [ ]:
from package.constants import *
from package.feature_extraction import *

In [ ]:
max_lags = 56

In [ ]:
interim = pd.read_parquet(interim_path)

In [ ]:
model = joblib.load(model_path)

In [ ]:
is_test = interim["demand"].isnull()
dates = interim.loc[is_test, "date"].unique()

In [ ]:
for date in tqdm(dates):
    is_date = (interim["date"] >= date - np.timedelta64(max_lags, "D")) & (
        interim["date"] <= date
    )

    intermediate = interim[is_date].copy()

    extract_features(intermediate)

    is_date = intermediate["date"] == date
    X = intermediate.loc[is_date, features]
    y_pred = model.predict(X)
    y_pred = np.maximum(0.0, y_pred)

    # See https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/141452
    y_pred *= 1.04

    is_date = interim["date"] == date
    interim.loc[is_date, "demand"] = y_pred

In [ ]:
interim[is_test].to_parquet(prediction_path)